## Clone OpenKE for benchmark datasets FB15K-237 and WN18

In [1]:
from pathlib import Path
import pickle

from tqdm import tqdm
import numpy as np
import random
from typing import List

import logging
# from dataclasses import dataclass
from typing import Dict

from tqdm import tqdm_notebook as tqdm
import pandas as  pd
import numpy as np
import traceback
import warnings
import random
import pickle

import torch
import torch.autograd
from torch import nn

from functools import partial
from typing import Optional, Union, List, Callable

# MyTorch imports
from mytorch.utils.goodies import *
from mytorch import dataiters

import wandb

## Prepare data

In [2]:
RAW_DATA_DIR = Path('./OpenKE/benchmarks/FB15K237')
PRETRAINING_DATA_DIR = Path('./data/pre_training_data')
np.random.seed(42)
random.seed(42)

In [3]:
# Make data iterator -> Modify Simple Iterator from mytorch
class QuintRankingSampler:
    """
        Given X and Y matrices (or lists of lists),
            it returns a batch worth of stuff upon __next__
    :return:
    """

    def __init__(self, data, bs: int = 64):

        self.bs = bs  # Batch Size (of neg items)
        self.pos = data["pos"]
        self.neg = data["neg"]

        self.initialize()

        self.n = len(self.pos)

        assert len(self.pos) == len(self.neg), "Mismatched lengths of pos and negs."

        # Shuffle everything
        self.shuffle()

    def initialize(self):
        """
            Right now, pos is a list of quints, neg is a list of list of quints.
            We want to flatten the negs; and repeat the pos
        """
        # Flatten negs
        flat_negs = []
        for negs in self.neg:
            flat_negs += negs

        self.neg_times = self.neg[0].__len__()

        # Repate pos
        repeat_pos = []
        for pos in self.pos:
            repeat_pos += [pos for _ in range(self.neg_times)]

        self.pos = np.array(repeat_pos)
        self.neg = np.array(flat_negs)

    def shuffle(self):
        """ Shuffle pos and neg together """

        shuffle_ids = np.arange(self.n)
        np.random.shuffle(shuffle_ids)
        #         print(shuffle_ids)
        self.pos = self.pos[shuffle_ids]
        self.neg = self.neg[shuffle_ids]

    def __len__(self):
        return self.n // self.bs - (1 if self.n * self.neg_times % self.bs else 0)

    def __iter__(self):
        self.i = 0
        return self

    def __next__(self):
        """
            Each time, take `bs` pos (and take one neg from each `bs` pos)
        """
        if self.i >= self.n:
            print("Should stop")
            raise StopIteration

        _pos = self.pos[self.i: min(self.i + self.bs, len(self.pos) - 1)]
        _neg = self.neg[self.i: min(self.i + self.bs, len(self.pos) - 1)]

        self.i = min(self.i + self.bs, self.n)

        return _pos, _neg


class SingleSampler:
    """ Another sampler which gives correct + all corrupted things for one triple """

    def __init__(self, data: dict, bs: int):
        self.data = {'pos': np.array(data['pos'], dtype=np.int), 'neg': np.array(data['neg'], dtype=np.int)}
        self.bs = bs

        assert len(self.data['pos']) == len(self.data['neg']), "Mismatched lengths between pos and neg data!"
        self.shuffle()

    def shuffle(self):
        shuffle_ids = np.arange(len(self.data['pos']))
        np.random.shuffle(shuffle_ids)

        self.data = {'pos': self.data['pos'][shuffle_ids], 'neg': self.data['neg'][shuffle_ids]}

    def __len__(self):
        return len(self.data['pos']) // self.bs

    def __iter__(self):
        self.i = 0
        return self

    def __next__(self):
        """ Concat pos quint with n neg quints (corresponding)"""
        if self.i >= self.data['pos'].__len__():
            print("Should stop")
            raise StopIteration

        res = np.zeros((self.bs, self.data['neg'][self.i].__len__() + 1, 3), dtype=np.int)
        pos = self.data['pos'][self.i: min(self.i + self.bs, len(self.data['pos']))]
        neg = self.data['neg'][self.i: min(self.i + self.bs, len(self.data['neg']))]
        for i, (_p, _n) in enumerate(zip(pos, neg)):
            res[i, 0] = _p
            res[i, 1:] = _n

        self.i += self.bs
        return res



In [4]:

training_triples = []
valid_triples = []
test_triples = []

with open(RAW_DATA_DIR / "entity2id.txt", "r") as ent_file, \
    open(RAW_DATA_DIR / "relation2id.txt", "r") as rel_file, \
    open(RAW_DATA_DIR / "train2id.txt", "r") as train_file, \
    open(RAW_DATA_DIR / "valid2id.txt", "r") as valid_file, \
    open(RAW_DATA_DIR / "test2id.txt", "r") as test_file:
    num_entities = int(next(ent_file).strip("\n"))
    num_relations = int(next(rel_file).strip("\n"))
    num_trains = int(next(train_file).strip("\n"))
    for line in train_file:
        triple = line.strip("\n").split(" ")
        training_triples.append([int(triple[0]), int(triple[2]), int(triple[1])])
        
    num_valid = int(next(valid_file).strip("\n"))
    for line in valid_file:
        triple = line.strip("\n").split(" ")
        valid_triples.append([int(triple[0]), int(triple[2]), int(triple[1])])
    
    num_test = int(next(test_file).strip("\n"))
    for line in test_file:
        triple = line.strip("\n").split(" ")
        test_triples.append([int(triple[0]), int(triple[2]), int(triple[1])])

In [5]:
EXPERIMENT_CONFIG = {
    'NUM_ENTITIES': num_entities,
    'NUM_RELATIONS': num_relations,
    'EMBEDDING_DIM': 200,
    'NORM_FOR_NORMALIZATION_OF_ENTITIES': 2,
    'NORM_FOR_NORMALIZATION_OF_RELATIONS': 2,
    'SCORING_FUNCTION_NORM': 1,
    'MARGIN_LOSS': 4,
    'LEARNING_RATE': 10,
    'NEGATIVE_SAMPLING_PROBS': [0.3, 0.0, 0.2, 0.5],
    'NEGATIVE_SAMPLING_TIMES': 10,
    'BATCH_SIZE': 8192,
    'EPOCHS': 100
}

In [6]:
def sample_negatives(triple: List) -> List:
    if np.random.random() < 0.5:
        # sample subject
        return [random.choice(range(num_entities)), triple[1], triple[2]]
    else:
        # sample object
        return [triple[0], triple[1], random.choice(range(num_entities))]   

In [7]:
def generate_negatives(positive: List[List], times: int):
    """
        :param postive: List of the raw data
        :param times: how many negative samples per positive sample.
    """
    negatives = []
    for pos in tqdm(positive):
        negatives_per_pos = [sample_negatives(pos) for _ in range(times)]
        negatives.append(negatives_per_pos)
        
    return negatives

In [8]:
try:
    negatives = pickle.load(open(PRETRAINING_DATA_DIR / 'fb15k_negatives.pkl', 'rb'))
except (FileNotFoundError, IOError) as e:
    # Generate it again
    warnings.warn("Negative data not pre-generating. Takes three minutes.")
    negatives = generate_negatives(training_triples + valid_triples, 
                                   times = EXPERIMENT_CONFIG['NEGATIVE_SAMPLING_TIMES'])

    # Dump this somewhere
    with open(PRETRAINING_DATA_DIR / 'fb15k_negatives.pkl', 'wb+') as f:
        pickle.dump(negatives, f)

In [9]:
train_neg = negatives[:len(training_triples)]
val_neg = negatives[len(training_triples):]

In [10]:
data = {'train': {'pos': training_triples, 'neg': train_neg}, 'valid': {'pos': valid_triples, 'neg': val_neg}}

In [11]:
training_triples[0:5], train_neg[0:5], valid_triples[0:5], val_neg[0:5]

([[0, 0, 1], [2, 1, 3], [4, 2, 5], [6, 3, 7], [8, 4, 9]],
 [[[10476, 0, 1],
   [0, 0, 1824],
   [0, 0, 409],
   [0, 0, 12149],
   [4506, 0, 1],
   [4012, 0, 1],
   [3657, 0, 1],
   [0, 0, 2286],
   [0, 0, 12066],
   [0, 0, 1679]],
  [[11087, 1, 3],
   [2, 1, 12135],
   [2, 1, 8935],
   [1424, 1, 3],
   [9674, 1, 3],
   [6912, 1, 3],
   [520, 1, 3],
   [2, 1, 488],
   [1535, 1, 3],
   [3582, 1, 3]],
  [[4, 2, 3811],
   [8279, 2, 5],
   [9863, 2, 5],
   [434, 2, 5],
   [9195, 2, 5],
   [4, 2, 3257],
   [11731, 2, 5],
   [4, 2, 10647],
   [4, 2, 11490],
   [8928, 2, 5]],
  [[6, 3, 6873],
   [3611, 3, 7],
   [7359, 3, 7],
   [6, 3, 9654],
   [6, 3, 4557],
   [6, 3, 13261],
   [14242, 3, 7],
   [106, 3, 7],
   [6, 3, 12432],
   [13202, 3, 7]],
  [[2615, 4, 9],
   [11438, 4, 9],
   [6924, 4, 9],
   [8, 4, 5574],
   [4552, 4, 9],
   [8, 4, 2547],
   [3527, 4, 9],
   [8, 4, 12509],
   [8, 4, 5514],
   [1674, 4, 9]]],
 [[6404, 31, 1211],
  [4089, 154, 1437],
  [8549, 85, 434],
  [8379, 47, 3498

## Model

In [12]:
class BaseModule(nn.Module):
    """A base class for all of the models."""

    margin_ranking_loss_size_average: bool = None
    entity_embedding_max_norm: Optional[int] = None
    entity_embedding_norm_type: int = 2
    hyper_params = [EXPERIMENT_CONFIG['EMBEDDING_DIM'], 
                    EXPERIMENT_CONFIG['MARGIN_LOSS'], 
                    EXPERIMENT_CONFIG['LEARNING_RATE']]

    def __init__(self, config: Dict) -> None:
        super().__init__()

        # Device selection
        self.device = config['DEVICE']

        # Loss
        self.margin_loss = config['MARGIN_LOSS']
        self.criterion = nn.MarginRankingLoss(
            margin=self.margin_loss,
            reduction='mean' if self.margin_ranking_loss_size_average else 'sum'
        )

        # Entity dimensions
        #: The number of entities in the knowledge graph
        self.num_entities = config['NUM_ENTITIES']
        #: The number of unique relation types in the knowledge graph
        self.num_relations = config['NUM_RELATIONS']
        #: The dimension of the embeddings to generate
        self.embedding_dim = config['EMBEDDING_DIM']

        self.entity_embeddings = nn.Embedding(
            self.num_entities,
            self.embedding_dim,
            norm_type=self.entity_embedding_norm_type,
            max_norm=self.entity_embedding_max_norm,
        )

    def __init_subclass__(cls, **kwargs):  # noqa: D105
        if not getattr(cls, 'model_name', None):
            raise TypeError('missing model_name class attribute')

    def _get_entity_embeddings(self, entities):
        return self.entity_embeddings(entities).view(-1, self.embedding_dim)

    def _compute_loss(self, positive_scores: torch.Tensor, negative_scores: torch.Tensor) -> torch.Tensor:
        y = np.repeat([1], repeats=positive_scores.shape[0])
        y = torch.tensor(y, dtype=torch.float, device=self.device)

        loss = self.criterion(positive_scores, negative_scores, y)
        return loss

In [13]:
def slice_triples(triples: torch.Tensor) -> List[torch.Tensor]:
    """ Slice in 3 or 5 as needed """
    return triples[:,0], triples[:,1], triples[:,2]

class TransE(BaseModule):
    """An implementation of TransE [borders2013]_.
     This model considers a relation as a translation from the head to the tail entity.
    .. [borders2013] Bordes, A., *et al.* (2013). `Translating embeddings for modeling multi-relational data
                     <http://papers.nips.cc/paper/5071-translating-embeddings-for-modeling-multi-relational-data.pdf>`_
                     . NIPS.
    .. seealso::
       - Alternative implementation in OpenKE: https://github.com/thunlp/OpenKE/blob/OpenKE-PyTorch/models/TransE.py
    """

    model_name = 'TransE MM'
    margin_ranking_loss_size_average: bool = True
    entity_embedding_max_norm: Optional[int] = None
    entity_embedding_norm_type: int = 2
    
    def __init__(self, config) -> None:
        super().__init__(config)

        # Embeddings
        self.l_p_norm_entities = config['NORM_FOR_NORMALIZATION_OF_ENTITIES']
        self.scoring_fct_norm = config['SCORING_FUNCTION_NORM']
        self.relation_embeddings = nn.Embedding(config['NUM_RELATIONS'], config['EMBEDDING_DIM'])
        
        self.config = config

        self._initialize()

    def _initialize(self):
        embeddings_init_bound = 6 / np.sqrt(self.config['EMBEDDING_DIM'])
        nn.init.uniform_(
            self.entity_embeddings.weight.data,
            a=-embeddings_init_bound,
            b=+embeddings_init_bound,
        )
        nn.init.uniform_(
            self.relation_embeddings.weight.data,
            a=-embeddings_init_bound,
            b=+embeddings_init_bound,
        )

        norms = torch.norm(self.relation_embeddings.weight, 
                           p=self.config['NORM_FOR_NORMALIZATION_OF_RELATIONS'], dim=1).data
        self.relation_embeddings.weight.data = self.relation_embeddings.weight.data.div(
            norms.view(self.num_relations, 1).expand_as(self.relation_embeddings.weight))

    def predict(self, triples):
        scores = self._score_triples(triples)
        return scores

    def forward(self, batch_positives, batch_negatives):
        # Normalize embeddings of entities
        norms = torch.norm(self.entity_embeddings.weight, p=self.l_p_norm_entities, dim=1).data
        self.entity_embeddings.weight.data = self.entity_embeddings.weight.data.div(
            norms.view(self.num_entities, 1).expand_as(self.entity_embeddings.weight))

        positive_scores = self._score_triples(batch_positives)
        negative_scores = self._score_triples(batch_negatives)
        loss = self._compute_loss(positive_scores=positive_scores, negative_scores=negative_scores)
        return (positive_scores, negative_scores), loss

    def _score_triples(self, triples):
        
        head_embeddings, relation_embeddings, tail_embeddings = self._get_triple_embeddings(triples)
        scores = self._compute_scores(head_embeddings, relation_embeddings, tail_embeddings)
        return scores

    def _compute_scores(self, head_embeddings, relation_embeddings, tail_embeddings):
        """
            Compute the scores based on the head, relation, and tail embeddings.
        
        :param head_embeddings: embeddings of head entities of dimension batchsize x embedding_dim
        :param relation_embeddings: emebddings of relation embeddings of dimension batchsize x embedding_dim
        :param tail_embeddings: embeddings of tail entities of dimension batchsize x embedding_dim
        :return: Tensor of dimension batch_size containing the scores for each batch element
        """
        # Add the vector element wise
        sum_res = head_embeddings + relation_embeddings - tail_embeddings 
        distances = torch.norm(sum_res, dim=1, p=self.scoring_fct_norm).view(size=(-1,))
        return distances

    def _get_triple_embeddings(self, triples):
        heads, relations, tails = slice_triples(triples)
        return (
            self._get_entity_embeddings(heads),
            self._get_relation_embeddings(relations),
            self._get_entity_embeddings(tails)
        )

    def _get_relation_embeddings(self, relations):
        return self.relation_embeddings(relations).view(-1, self.embedding_dim)

## Training

In [14]:
config = EXPERIMENT_CONFIG.copy()
config['DEVICE'] = torch.device('cuda')
model = TransE(config)
model.to(config['DEVICE'])
optimizer = torch.optim.SGD(model.parameters(), lr=config['LEARNING_RATE'])


wandb.init(project="wikidata-embeddings")
for k, v in config.items():
    wandb.config[k] = v

In [15]:
def evaluate_pointwise(pos_scores: torch.Tensor, neg_scores:torch.Tensor)->torch.Tensor:
    """
        Given a pos and neg quint, how many times did the score for positive be more than score for negative
    
        :param pos_scores: scores corresponding to pos quints (bs, )
        :param neg_scores: scores corresponding to neg quints (bs, )
        :return accuracy (0d tensor)
    """
    return torch.mean((pos_scores>=neg_scores).float()).item()
    
def evaluate_dataset(scores:torch.Tensor):
    """
        Compute score for `bs` set of [pos, neg, neg .....] quints.
        Assume pos is at the first position.
        
        
        :param scores: torch tensor of scores (bs,neg_samples+1)
        :returns (acc, mrr) both 1d tensors.
    """
    accuracy = (torch.argmax(scores, dim=1)==0).float()
    ranks = (torch.argsort(-scores, dim=1) == 0).nonzero()[:,1]
    print(ranks)
    recirank = 1.0/(ranks+1).float()
    
    return accuracy.detach().cpu().numpy(), recirank.detach().cpu().numpy()

In [16]:
# Make a loop fn
def simplest_loop(epochs: int,
                  data: dict,
                  opt: torch.optim,
                  train_fn: Callable,
                  predict_fn: Callable,
                  device: torch.device = torch.device('cpu'),
                  data_fn: Callable = dataiters.SimplestSampler,
                  data_fn_val: Callable = dataiters.SimplestSampler,
                  eval_fn_trn: Callable = default_eval,
                  eval_fn_val: Callable = default_eval) -> (list, list, list):
    """
        A fn which can be used to train a language model.

        The model doesn't need to be an nn.Module,
            but have an eval (optional), a train and a predict function.

        Data should be a dict like so:
            {"train":{"x":np.arr, "y":np.arr}, "val":{"x":np.arr, "y":np.arr} }

        Train_fn must return both loss and y_pred

        :param epochs: number of epochs to train for
        :param data: a dict having keys train_x, test_x, train_y, test_y
        :param device: torch device to create new tensor from data
        :param opt: optimizer
        :param loss_fn: loss function
        :param train_fn: function to call with x and y
        :param predict_fn: function to call with x (test)
        :param data_fn: a class to which we can pass training data and get an iterator.
        :param data_fn_val: can be same as above; or diff. Specifically for validation runs.
        :param eval_fn: (optional) function which when given pred and true, returns acc
        :return: traces
    """

    train_loss = []
    train_acc = []
    valid_acc = []
    valid_acc_like_trn = []
    valid_mrr = []
    lrs = []

    # Epoch level
    for e in range(epochs):

        per_epoch_loss = []
        per_epoch_tr_acc = []

        # Train
        with Timer() as timer:

            # Make data
            trn_dl, val_dl_like_trn, val_dl = data_fn(data['train']), data_fn(data['valid']), data_fn_val(data['valid'])

            for pos, neg in tqdm(trn_dl):
                opt.zero_grad()

                _pos = torch.tensor(pos, dtype=torch.long, device=device)
                _neg = torch.tensor(neg, dtype=torch.long, device=device)

                (pos_scores, neg_scores), loss = train_fn(_pos, _neg)

                per_epoch_tr_acc.append(eval_fn_trn(pos_scores=pos_scores, neg_scores=neg_scores))
                per_epoch_loss.append(loss.item())

                loss.backward()
                opt.step()

        """
            # Val
            Run through the dataset twice.
                1. same as training data (pointwise eval)
                2. One quint (pos+negs) at a time. 
        """ 
        
        with torch.no_grad():

            per_epoch_vl_acc, per_epoch_vl_mrr, per_epoch_vl_acc_like_trn = [], [], []
            for quints in tqdm(val_dl):
                _quints = torch.tensor(quints, dtype=torch.long, device=device)
                
                # Flatten it
                _quints_shape = _quints.shape
                scores = predict_fn(_quints.view(-1, _quints_shape[-1]))
                scores = scores.view(_quints_shape[0], _quints_shape[1])
                
                accuracy, recirank = eval_fn_val(scores)

                per_epoch_vl_acc.append(accuracy)
                per_epoch_vl_mrr.append(recirank)
            
            for pos, neg in tqdm(val_dl_like_trn):
                
                _pos = torch.tensor(pos, dtype=torch.long, device=device)
                _neg = torch.tensor(neg, dtype=torch.long, device=device)

                (pos_scores, neg_scores), loss = train_fn(_pos, _neg)
                acc = eval_fn_trn(pos_scores=pos_scores, neg_scores=neg_scores)
                
                per_epoch_vl_acc_like_trn.append(acc)

        # Bookkeep
        train_acc.append(np.mean(per_epoch_tr_acc))
        train_loss.append(np.mean(per_epoch_loss))
        valid_acc.append(np.mean(per_epoch_vl_acc))
        valid_acc_like_trn.append(np.mean(per_epoch_vl_acc_like_trn))
        valid_mrr.append(np.mean(per_epoch_vl_mrr))
        
        print("Epoch: %(epo)03d | Loss: %(loss).5f | Tr_c: %(tracc)0.5f | Vl_c: %(vlacc)0.5f | Vl_c_liketrn: %(vlacc_liketrn)0.5f | Vl_mrr: %(vlmrr)0.5f |Time: %(time).3f min"
              % {'epo': e,
                 'loss': float(np.mean(per_epoch_loss)),
                 'tracc': float(np.mean(per_epoch_tr_acc)),
                 'vlacc': float(np.mean(per_epoch_vl_acc)),
                 'vlacc_liketrn': float(np.mean(per_epoch_vl_acc_like_trn)),
                 'vlmrr': float(np.mean(per_epoch_vl_mrr)),
                 'time': timer.interval / 60.0})

        # Wandb stuff
        wandb.log({
            'epoch': e, 
            'loss': float(np.mean(per_epoch_loss)),
            'trn_acc': float(np.mean(per_epoch_tr_acc)),
            'val_acc': float(np.mean(per_epoch_vl_acc)),
            'val_acc_liketrn': float(np.mean(per_epoch_vl_acc_like_trn)),
            'val_mrr': float(np.mean(per_epoch_vl_mrr))
        })

    return train_acc, valid_acc, valid_acc_like_trn, valid_mrr, train_loss

In [17]:
args = {
    "epochs":config['EPOCHS'],
    "data":data,
    "opt": optimizer,
    "train_fn": model,
    "predict_fn": model.predict,
    "device": config['DEVICE'],
    "data_fn": partial(QuintRankingSampler, bs=config["BATCH_SIZE"]),
    "data_fn_val": partial(SingleSampler, bs=config["BATCH_SIZE"]),
    "eval_fn_trn": evaluate_pointwise,
    "eval_fn_val": evaluate_dataset
}

In [18]:
simplest_loop(**args)

Should stop



tensor([0, 0, 1,  ..., 3, 0, 0], device='cuda:0')
tensor([0, 1, 8,  ..., 1, 0, 1], device='cuda:0')
tensor([0, 0, 1,  ..., 5, 5, 5], device='cuda:0')
Should stop



Should stop

Epoch: 000 | Loss: 1.45354 | Tr_c: 0.88471 | Vl_c: 0.73674 | Vl_c_liketrn: 0.91650 | Vl_mrr: 0.59709 |Time: 0.094 min


Should stop



tensor([1, 0, 0,  ..., 1, 3, 7], device='cuda:0')
tensor([2, 0, 0,  ..., 0, 3, 3], device='cuda:0')
tensor([2, 1, 0,  ..., 5, 5, 5], device='cuda:0')
Should stop



...

Should stop

Epoch: 078 | Loss: 0.21314 | Tr_c: 0.99084 | Vl_c: 0.76786 | Vl_c_liketrn: 0.92845 | Vl_mrr: 0.61912 |Time: 0.082 min


Should stop



tensor([0, 0, 0,  ..., 0, 2, 0], device='cuda:0')
tensor([ 0,  0,  1,  ...,  2, 10,  0], device='cuda:0')
tensor([0, 0, 1,  ..., 5, 5, 5], device='cuda:0')
Should stop



Should stop

Epoch: 079 | Loss: 0.21291 | Tr_c: 0.99081 | Vl_c: 0.76819 | Vl_c_liketrn: 0.92825 | Vl_mrr: 0.61907 |Time: 0.083 min


KeyboardInterrupt: 